 ![](fig/python-logo.png)

### Master 2 DUAS Unistra,  2017-2018

***

# Programmer en Python
***

## Une introduction à Pandas

***

Matthieu Boileau - [matthieu.boileau@math.unistra.fr](mailto: matthieu.boileau@math.unistra.fr)


*Contenu sous licence [CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0), fortement inspiré de <https://github.com/pnavaro/big-data>*

![pandas](http://pandas.pydata.org/_static/pandas_logo.png "Pandas Logo")

# Pandas 

Un outil pour l'analyse de données

- première version en 2011
- basé sur NumPy
- largement inspiré par la toolbox R pour la manipulation de données
- structures de données auto-descriptives
- Fonctions de chargement et écriture vers les formats de fichiers courants
- Fonctions de tracé
- Outils statistiques basiques


# Les *Pandas series*

[Documentation officielle](https://pandas.pydata.org/pandas-docs/stable/dsintro.html#series)

- Une *series* Pandas:
    - un tableau 1D de données (éventuellement hétérogènes)
    - une séquence d'étiquettes appelée *index* de même longueur que le tableau 1D
    
- l'index peut être du contenu numérique, des chaînes de caractères, ou des dates-heures.
- si l'index est une valeur temporelle, alors il s'agit d'une [*time series*](https://en.wikipedia.org/wiki/Time_series)
- l'index par défaut est `range(len(data))`

## Illustration

In [ ]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", 8)  # Pour limiter le nombre de lignes affichées

In [ ]:
print(pd.Series([10, 8, 7, 6, 5]))
print(pd.Series([4, 3, 2, 1, 0.]))

## Une série temporelle

Par exemple, les jours qui nous séparent du nouvel an.

In [ ]:
time_period = pd.period_range('12/12/2017', '01/01/2018', freq="D")
pd.Series(index=time_period, data=range(len(time_period) - 1, -1, -1)) 

## Un exemple de traitement

On exploite un texte tiré de ce site non officiel : http://www.sacred-texts.com/neu/mphg/mphg.htm

In [ ]:
with open("exos/nee.txt") as f:
    nee = f.read()

print(nee)

### On veut dénombrer les occurrences de mots

On copie la chaîne initiale (par respect pour l'auteur)

In [ ]:
nee2 = nee

On supprime la ponctuation 

In [ ]:
for s in '.', '!', ',', '?', ':', '[', ']':
    nee2 = nee2.replace(s, '')

On transforme en minuscule et on découpe en une liste de mots

In [ ]:
nee2 = nee2.lower().split()

On crée un object compteur

In [ ]:
from collections import Counter
c = Counter(nee2)

On ne retient que les mots qui apparaissent plus de 2 fois

In [ ]:
c = Counter({x : c[x] for x in c if c[x] > 2})
c

### On crée une série Pandas à partir de l'objet  `c`

> Notons que la série est ordonnée avec un index croissant (dans l'ordre alphabétique).

In [ ]:
words = pd.Series(c)
words

### On représente cette série sur un histogramme

On commence par positionner certains paramètres de tracé

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Pour un rendu plus abouti https://seaborn.pydata.org/introduction.html
import seaborn as sns  
sns.set()

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (9, 6)  # Pour obtenir des figures plus grandes

In [ ]:
words.plot(kind='bar');

## Indexation et slicing

L'indexation et le slicing est une sorte de mélange entre les listes et les dictionnaires :

- `series[index]` pour accéder à la donnée correspondant à `index`
- `series[i]` où `i` est un entier qui suit les règles de l'indexation en python

Nombre d'occurrences de la chaîne `nee`

In [ ]:
print(words.index)  # Pour rappel
words["nee"]

Trois dernières données de la série

In [ ]:
words[-3:]

## On veut ordonner la série



In [ ]:
words.sort_values(inplace=True)
words.plot(kind='barh');  # On change pour un histogramme horizontal

# Les *Pandas Dataframes*

- C'est la structure de base de Pandas
- un *Dataframe* est une structure de données tabulées à deux dimensions, potentiellement hétérogène
- un *Dataframe* est constitué de lignes et colonnes portant des étiquettes
- C'est un peu un "dictionnaire de *Series*".

## Un exemple avec les arbres de la ville de Strasbourg

Conformément à l'[ordonnance du 6 juin 2005](https://www.legifrance.gouv.fr/affichTexte.do;jsessionid=0498736F8F3B3936EDA857F095A5434A.tpdjo08v_3?cidTexte=JORFTEXT000000629684&categorieLien=id) (qui prolonge la loi CADA), la ville de Strasbourg a commencé à mettre en ligne ses données publiques.

En particulier des données sur ses arbres : https://www.strasbourg.eu/arbres-alignements-espaces-verts

On veut exploiter ces données. Pour ce faire, on va :

1. télécharger les données
2. les charger dans un *Dataframe*
3. les nettoyer/filtrer
4. les représenter graphiquement

On commence par définir une fonction qui télécharge et extrait une archive zip.

In [ ]:
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

def download_unzip(zipurl, destination):
    """Download zipfile from URL and extract to destination"""
    with urlopen(zipurl) as zipresp:
        with ZipFile(BytesIO(zipresp.read())) as zfile:
            zfile.extractall(destination)

On l'utilise pour télécharger l'archive des données ouvertes de la ville de Strasbourg.

In [ ]:
download_unzip("https://www.strasbourg.eu/documents/976405/1168331/CUS_CUS_DEPN_ARBR.zip", "arbres")

On liste le contenu de l'archive

In [ ]:
%ls -R arbres

On charge le fichier csv comme un *Dataframe*.

In [ ]:
arbres_all = pd.read_csv("arbres/CUS_CUS_DEPN_ARBR.csv", encoding='latin', delimiter=";")
arbres_all

In [ ]:
print("{} arbres recensés !".format(len(arbres_all)))

On commence par lister les villes citées.

In [ ]:
print(set(arbres_all['point vert VILLE']))

On ne s'intéresse qu'à la ville de Strasbourg

In [ ]:
arbres = arbres_all[arbres_all['point vert VILLE'] ==  "STRASBOURG"]
print("Il ne reste plus que {} arbres.".format(len(arbres)))

On enlève les données incomplètes.

In [ ]:
arbres = arbres.dropna(axis=0, how='any')
print("Il ne reste plus que {} arbres.".format(len(arbres)))

On extrait la série des essences.

In [ ]:
essences = set(arbres['Libellé_Essence'])
print("Il y a {} essences différentes !".format(len(essences)))

Les 5 premières dans l'ordre alphabétique :

In [ ]:
sorted(list(essences))[:5]

C'est bientôt Noël, on se limite aux sapins !

In [ ]:
sapins = arbres[arbres['Libellé_Essence'].str.match("^Abies")]
sapins

On trace leur répartition

In [ ]:
sapins['Libellé_Essence'].value_counts().plot("barh");

# Représentation géographique

On voudrait maintenant représenter la répartition des arbres par quartiers.

On utilise à nouveau les données ouvertes de la ville de Strasbourg, cette fois-ci concernant les quartiers : https://www.strasbourg.eu/decoupage-15-quartiers

On télécharge, on extrait l'archive et on liste son contenu.

In [ ]:
download_unzip("https://www.strasbourg.eu/documents/976405/1168339/CUS_CUS_DUAH_QUART.zip", "quartiers")
%ls -R quartiers

C'est le fichier `.shp` qui nous intéresse.

À ce stade, nous avons besoin des bibliothèques [GeoPandas](http://geopandas.org/) et [Folium](https://folium.readthedocs.io/en/latest/) que l'on installe avec pip.

On commence par installer pip, le gestionnaire de paquets Python.

In [ ]:
from urllib.request import urlretrieve

urlretrieve("https://bootstrap.pypa.io/get-pip.py", "get-pip.py")
%run get-pip.py

On installe les paquets nécessaires dans le kernel python courant.

In [ ]:
import sys
!{sys.executable} -m pip install geopandas folium

On charge le fichier qui nous intéresse.

In [ ]:
import geopandas as gpd
quartiers = gpd.read_file("quartiers/SHP/Quartiers_Strasbourg_15.shp")
print("quartiers est de type {}.".format(type(quartiers)))
quartiers

Avec Folium, on commence par représenter ces données géographiques sur un fond de cartes.

In [ ]:
import folium

# On crée une carte initialement centrée sur Strasbourg
STRASBOURG_COORD = (48.58, 7.75)
stras_map = folium.Map(STRASBOURG_COORD, zoom_start=11, tiles='cartodbpositron')

# On ajoute les données des quartiers
folium.GeoJson(quartiers).add_to(stras_map)

# On enregistre dans un fichier html
stras_map.save('stras_map.html')

# On trace dans le notebook
display(stras_map)

À l'emplacement de ces quartiers, on souhaite représenter une échelle de couleur en fonction de la densité d'arbres.

On constate que les noms de quartiers sont différents de ceux du jeu de données sur les arbres.

In [ ]:
print(set(quartiers["QUARTIER"]), len(quartiers["QUARTIER"]))
print(set(arbres['Point vert Quartier usuel']), len(set(arbres['Point vert Quartier usuel'])))

On commence par mettre en minuscule les noms contenus dans le Dataframe `quartiers` et à remplacer les espaces par des underscores

In [ ]:
quartiers["QUARTIER"] = quartiers["QUARTIER"].str.lower()
quartiers["QUARTIER"] = quartiers["QUARTIER"].str.replace('_', ' ')

À présent, on  convertit les noms dans le Dataframe `arbres` en supposant les correspondances ci-dessous.

In [ ]:
convertion_dict = {"CENTRE": "centre ville",
                   ("BOURSE", "ESPLANADE", "KRUTENAU"): "bourse esplanade krutenau",
                   ("ORANGERIE", "CONSEIL-XV"): "orangerie conseil des xv",
                   ("GARE", "TRIBUNAL"): "gare tribunal",
                   ("HAUTEPIERRE", "POTERIE"): "hautepierre poteries",
                   "MUSAU": "NEUDORF",
                   "STOCKFELD": "NEUHOF2",
                   "PLAINE DES BOUCHERS": "MEINAU",
                   "POLYGONE": "NEUHOF",
                   "PORTE DE SCHIRMECK": "ELSAU",
                   ("ROBERTSAU", "WACKEN"): "ROBERTSAU WACKEN"}

for k, v in convertion_dict.items():
    arbres['Point vert Quartier usuel'] = arbres['Point vert Quartier usuel'].replace(to_replace=k, value=v)

arbres['Point vert Quartier usuel'] = arbres['Point vert Quartier usuel'].str.lower()
arbres

On vérifie que l'ensemble des quartiers est le même pour les deux Dataframes `quartiers` et `arbres`.

In [ ]:
set(quartiers["QUARTIER"]) == set(arbres['Point vert Quartier usuel'])

On construit une série qui contient le nombre d'arbres par quartier.

In [ ]:
arbres_quartiers = arbres['Point vert Quartier usuel'].value_counts()

On trace le graphique en barres correspondant.

In [ ]:
arbres_quartiers.plot(kind='barh');

On construit une nouvelle *Series* correspondant à l'aire de chaque quartier en $m^2$.

In [ ]:
aires = quartiers.area
aires.index = quartiers["QUARTIER"]

On calcule la densité d'arbres par hectare.

In [ ]:
densite = arbres_quartiers/aires*10000
densite

On trace une carte colorée par la densité d'arbres avec la méthode `choropleth`.

In [ ]:
stras_map.choropleth(geo_data=quartiers, 
             data=densite,
             key_on='feature.properties.QUARTIER',
             fill_color='YlGn',
             fill_opacity=0.5,
             line_opacity=0.2,
             legend_name=r"Nombre d\'arbres par hectare")
stras_map.save('stras_tree.html')
display(stras_map)

## Exercice

Ecrire une fonction qui prend en argument une essence d'arbres et qui trace le nombre d'arbres correspondant par quartier en utilisant `choropleth`.

<button data-toggle="collapse" data-target="#plot_essence" class='btn btn-primary'>Solution</button>
<div id="plot_essence" class="collapse">

```python
def plot_essence(essence_name):
    # On extrait un Dataframe ne contenant que l'essence
    essence = arbres[arbres['Libellé_Essence'].str.match("^{}".format(
                essence_name))]
    # On crée une Series contenant contenant le nombre d'arbres par quartier
    essence_quartiers = essence['Point vert Quartier usuel'].value_counts()

    # On crée une carte initialement centrée sur Strasbourg
    STRASBOURG_COORD = (48.58, 7.75)
    stras_map = folium.Map(STRASBOURG_COORD, zoom_start=11,
                           tiles='cartodbpositron')

    # On ajoute les données des quartiers
    folium.GeoJson(quartiers).add_to(stras_map)

    legend = "Nombre de {}".format(essence_name)
    stras_map.choropleth(geo_data=quartiers,
                         data=essence_quartiers,
                         key_on='feature.properties.QUARTIER',
                         fill_color='YlGn',
                         fill_opacity=0.5,
                         line_opacity=0.2,
                         legend_name=legend)
    stras_map.save('stras_tree_essence.html')
    display(stras_map)
```

In [ ]:
#plot_essence("Acer")